In [ ]:
# !pip install tensorflow==1.15.0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.contrib.layers import conv2d, avg_pool2d, max_pool2d
from tensorflow.contrib.layers import batch_norm, l2_regularizer
print(tf.__version__)

In [ ]:
# getting MNIST dataset

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
import cv2

In [ ]:
# parameters of network

img_height = 28
img_width = 28
img_channel = 1
batch_size = 64
max_iter = 8000
lr = 0.1

In [ ]:
num_classes = 10

In [ ]:
# defining CNN architecture

def simpleNN(inputs, num_classes):
    with tf.variable_scope("simpleNN"):
        net = conv2d(inputs, 64, [7, 7], stride=2, scope='conv1')
        print(net.name, net.get_shape().as_list())
        net = conv2d(inputs, 64, [7, 7], stride=2, scope='conv2')
        print(net.name, net.get_shape().as_list())
        net = max_pool2d(net, [3, 3], stride=2, scope='maxpool1')
        print(net.name, net.get_shape().as_list())
        net = conv2d(inputs, 32, [7, 7], stride=2, scope='conv4')
        print(net.name, net.get_shape().as_list())
        net = conv2d(inputs, 32, [7, 7], stride=2, scope='conv5')
        net = max_pool2d(net, [3, 3], stride=2, scope='maxpool6')
        print(net.name, net.get_shape().as_list())
        net = conv2d(net, num_classes, [1, 1], stride=1, scope='conv7')
        print(net.name, net.get_shape().as_list())
        net_shape = net.get_shape().as_list()
#         net = avg_pool2d(net, [13, 13], stride=1, scope='avgpool10')
        net = avg_pool2d(net, [net_shape[1],net_shape[1]], stride=1, scope='avgpool8')
        print(net.name, net.get_shape().as_list())
        
        net = tf.squeeze(net, [2], name='logits')
        print(net.name, net.get_shape().as_list())
        return net


In [ ]:
# define a function for iterating through mini-batches of the data
def batch_generator(n_iter, X, y,  batch_size=16, 
                    shuffle=False, random_seed=None):
    batch_nos = [i for i in range(0, X.shape[0], batch_size)]
    i = int((n_iter%len(batch_nos)))
#     print("batch : ", i)
    if shuffle:  # shuffles sample order in output batch
        
        i = np.random.choice(batch_nos)
#     print("batch : ", i)
    return X[i:i+batch_size, :], y[i:i+batch_size]

In [ ]:
# Initialize placeholders 
x = tf.placeholder(dtype = tf.float32, shape = [None, img_height, img_width, img_channel])
y = tf.placeholder(dtype = tf.int32, shape = [None,1])
# logits = tf.placeholder(dtype = tf.int32, shape = [None,1,1,num_classes])


logits = simpleNN(x, num_classes)
# Define a loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                    logits = logits))
# Define an optimizer 
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

# # Convert logits to label indexes
# correct_pred = tf.argmax(logits, 2)
# print(correct_pred.get_shape())
# # Define an accuracy metric
# accuracy = tf.compat.v1.metrics.accuracy(tf.cast(correct_pred, tf.float32), y)

In [ ]:
# training and testing simpleCNN


tf.set_random_seed(1234)
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(max_iter):
#         print('EPOCH', i)
        input_images, input_labels = batch_generator(i, x_train, y_train, batch_size, shuffle=True)
#         print(input_images.shape, input_labels.shape)
        input_images = np.resize(input_images,(input_images.shape[0], input_images.shape[1], input_images.shape[2],1))
        input_labels = np.resize(input_labels, (input_labels.shape[0],1))
        logits1, op, iter_loss = sess.run([logits, train_op, loss], feed_dict={x: input_images, y: input_labels})
    
        if i % 10 == 0:
#             print(input_images.shape)
#             print(np.array(logits1).shape)
            pred = np.argmax(logits1, 2)
#             print("pred : ", pred)
#             print("logits1 : ", logits1)
#             print("actual : ", input_labels)
            acc= accuracy_score(input_labels, pred)
            print("Loss: ", iter_loss, "acc : ", acc)
saver = tf.train.Saver()
saver.save(sess, "./model_ckpt/simpleNN.ckpt")
#         print('DONE WITH EPOCH')


pred_test = []
for i in range(x_test.shape[0]):
    logits1 = sess.run([logits], feed_dict={x: np.resize(x_test[i], (1,img_height, img_width, img_channel))})
#     print(np.array(logits1).shape)
#     print(np.resize(test_images[i], (1,img_height, img_width, img_channel)).shape)
    pred = np.argmax(logits1,3)
    pred_test.append(pred[0][0][0])
acc= accuracy_score(y_test, pred_test)
print("test accuracy : ", acc)
sess.close()